In [16]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import os

## Merge Review Tables

In [17]:
review_fold = "./Scraper/ReviewScraper/"
tables_fold = review_fold + "tables/"
files = os.listdir(tables_fold)
files = [f for f in files if ".csv" in f]

In [18]:
tables = []
for f in files:
    tab = pd.read_csv(tables_fold + f)
    comp = f.split("_")[0].lower()
    tab['company'] = [comp] * len(tab)
    tables.append(tab)

In [19]:
# merge tables
merge_table = pd.concat(tables, ignore_index=True, sort=False)
# drop duplicates
merge_table = merge_table.drop_duplicates()
merge_table = merge_table.reset_index(drop=True)

In [20]:
print("Total samples in merged table: {}".format(len(merge_table)))

Total samples in merged table: 6066


In [21]:
merge_table.head()

,date,employee_title,location,employee_status,review_title,years_at_company,helpful,pros,cons,advice_to_mgmt,rating_overall,rating_balance,rating_culture,rating_career,rating_comp,rating_mgmt,company
0,Sat Nov 02 2019 17:40:06 GMT-0700 (Pacific Day...,Software Engineer,"San Francisco, CA",Current Employee,Good company,I have been working at Dropbox full-time for l...,NaN,"Good food(the best part), good environment,",Product is not competitive\nManagement needs s...,NaN,5.0,5.0,5.0,5.0,5.0,5.0,dropbox
1,Mon Sep 23 2019 20:40:55 GMT-0700 (Pacific Day...,Account Executive,"Austin, TX",Current Employee,Great place to work,I have been working at Dropbox full-time for m...,NaN,In-house chefs serving premium breakfast/lunch...,no cons specific to Dropbox,NaN,5.0,5.0,5.0,5.0,5.0,5.0,dropbox
2,Mon Oct 14 2019 14:08:26 GMT-0700 (Pacific Day...,Research,"San Francisco, CA",Current Employee,Avoid this place,I have been working at Dropbox full-time for l...,7.0,1. Food; hands down best food in tech. \n2. We...,"1. Poor leadership; director+ are indecisive, ...",Hire more experienced industry leaders.,1.0,1.0,1.0,1.0,2.0,1.0,dropbox
3,Fri Sep 13 2019 10:19:38 GMT-0700 (Pacific Day...,Manager,NaN,Former Employee,great place to work,I worked at Dropbox full-time for less than a ...,NaN,"everything, culture, benefit, people, work lif...",growing company comes with growing pain.,NaN,5.0,5.0,5.0,4.0,5.0,5.0,dropbox
4,Tue Sep 10 2019 15:34:52 GMT-0700 (Pacific Day...,Senior Software Engineer,"San Francisco, CA",Current Employee,Great place to work and be creative,I have been working at Dropbox full-time for l...,NaN,"Really appreciates you as an employee. Perks, ...",Management is young and inexperienced. It show...,NaN,5.0,5.0,5.0,4.0,5.0,3.0,dropbox


In [22]:
merge_table.to_csv(review_fold + "merged_reviews_table.csv", index=False)

## Merge Salary Tables for full time and Intern

In [23]:
salary_fold = "./Scraper/SalaryScraper/"
tables_fold = salary_fold + "tables/"
files = os.listdir(tables_fold)
files = [f for f in files if ".csv" in f]

In [24]:
tables = []
for f in files:
    tab = pd.read_csv(tables_fold + f)
    comp = f.split("-")[0].lower()
    tab['company'] = [comp] * len(tab)
    tables.append(tab)
    
# merge tables
merge_table = pd.concat(tables, ignore_index=True, sort=False)
# drop duplicates
merge_table = merge_table.drop_duplicates()
# remove errors due to scraping
merge_table = merge_table[merge_table["jobTitle"] != "California"]
merge_table = merge_table[merge_table["jobTitle"] != "CA"]
merge_table = merge_table.reset_index(drop=True)

merge_table.rename(columns={"jobTitle": "job_title", "meanPay": "mean_pay", "Range": "salary_range"}, inplace=True)

print("Total samples in merged table: {}".format(len(merge_table)))

Total samples in merged table: 7805


In [25]:
merge_table.head()

,job_title,mean_pay,salary_range,company
0,Software Engineer,"$114,610/yr",Range: $70K - $166K,cisco
1,Software Engineer IV,"$133,178/yr",Range: $99K - $169K,cisco
2,Software Engineer III,"$113,381/yr",Range: $79K - $162K,cisco
3,Network Consulting Engineer,"$115,899/yr",Range: $50K - $259K,cisco
4,Senior Software Engineer,"$134,594/yr",Range: $87K - $190K,cisco


In [26]:
def check_for_full_time(row):
    x, y = row['job_title'], row['mean_pay']
    keywords = ["Hourly", "Hour", "Intern", "/hr", "Internship"]
    for k in keywords:
        if k in x or k in y:
            return False
        
    return True

In [27]:
merge_table['full_time'] = merge_table.apply(check_for_full_time, axis=1)
merge_table_intern = merge_table[~merge_table['full_time']]
merge_table = merge_table[merge_table['full_time']]

merge_table = merge_table.reset_index(drop=True)
merge_table_intern = merge_table_intern.reset_index(drop=True)

In [28]:
merge_table.drop(['full_time'], axis=1, inplace=True)
merge_table_intern.drop(['full_time'], axis=1, inplace=True)

In [29]:
print("Total samples in merged table for full-time: {}".format(len(merge_table)))
print("Total samples in merged table for intern: {}".format(len(merge_table_intern)))

Total samples in merged table for full-time: 6322
Total samples in merged table for intern: 1483


In [30]:
merge_table.to_csv(salary_fold + "fulltime_merged_salaries_company_table.csv", index=False)

In [31]:
merge_table_intern.to_csv(salary_fold + "intern_merged_salaries_company_table.csv", index=False)

## Create category label

In [32]:
merge_table.head()

,job_title,mean_pay,salary_range,company
0,Software Engineer,"$114,610/yr",Range: $70K - $166K,cisco
1,Software Engineer IV,"$133,178/yr",Range: $99K - $169K,cisco
2,Software Engineer III,"$113,381/yr",Range: $79K - $162K,cisco
3,Network Consulting Engineer,"$115,899/yr",Range: $50K - $259K,cisco
4,Senior Software Engineer,"$134,594/yr",Range: $87K - $190K,cisco


In [33]:
def generate_category_for_fulltime(y):
    y = y.lower()
    x = y.split(" ")
    sales = ["sales"]
    marketing = ["marketing", "market"]
    it = ["it"]
    enginer = ["software", "engineer", "engineering", "ui", "scientist", "developer", "sde1", "sde2"]
    manager = ["manager"]

    if any(keyw in x for keyw in sales): # 686
        return "sales"
    if any(keyw in x for keyw in marketing): # 289
        return "marketing"
    if any(keyw in x for keyw in it): # 136
        return "it"
    if any(keyw in x for keyw in enginer) or "member of technical staff" in y:
        return "engineering"
    if any(keyw in x for keyw in manager):
        return "managerial"
    
    return "others"

In [34]:
merge_table['category'] = merge_table['job_title'].apply(generate_category_for_fulltime)

In [35]:
merge_table.head()

,job_title,mean_pay,salary_range,company,category
0,Software Engineer,"$114,610/yr",Range: $70K - $166K,cisco,engineering
1,Software Engineer IV,"$133,178/yr",Range: $99K - $169K,cisco,engineering
2,Software Engineer III,"$113,381/yr",Range: $79K - $162K,cisco,engineering
3,Network Consulting Engineer,"$115,899/yr",Range: $50K - $259K,cisco,engineering
4,Senior Software Engineer,"$134,594/yr",Range: $87K - $190K,cisco,engineering


In [36]:
merge_table.to_csv(salary_fold + "fulltime_merged_salaries_company_table.csv", index=False)

In [37]:
def generate_category_for_intern(y):
    y = y.lower()
    x = y.split(" ")
    sales = ["sales"]
    marketing = ["marketing", "market"]
    it = ["it"]
    enginer = ["software", "engineer", "engineering", "ui", "scientist", "developer", "sde1", "sde2"]
    manager = ["manager"]

    if any(keyw in x for keyw in sales): # 686
        return "sales"
    if any(keyw in x for keyw in marketing): # 289
        return "marketing"
    if any(keyw in x for keyw in it): # 136
        return "it"
    if any(keyw in x for keyw in enginer) or "member of technical staff" in y:
        return "engineering"
    if any(keyw in x for keyw in manager):
        return "managerial"
    
    return "others"

In [38]:
merge_table_intern["category"] = merge_table_intern["job_title"].apply(generate_category_for_fulltime)

In [39]:
merge_table_intern.head()

,job_title,mean_pay,salary_range,company,category
0,Software Engineer - Hourly Intern,$28/hr,Range: $13 - $42,cisco,engineering
1,Software Engineer() - Hourly Intern,$29/hr,Range: $21 - $43,cisco,engineering
2,Intern - Hourly,$26/hr,Range: $13 - $45,cisco,others
3,Intern - Hourly,$24/hr,Range: $11 - $43,cisco,others
4,IT Engineer - Hourly Intern,$27/hr,Range: $24 - $30,cisco,it


In [40]:
merge_table_intern.to_csv(salary_fold + "intern_merged_salaries_company_table.csv", index=False)